# Compute Mo from strain rate model for all sources
This notebook computes for all the area sources in a model the scalar seismic moment. 

In [ ]:
%matplotlib inline
import os
import re
import sys
import h5py

from rtree import index
from oqmbt.oqt_project import OQtProject
from oqmbt.tools.strain import get_moment_for_polygon
from oqmbt.tools.mfd import get_moment_from_mfd

In [ ]:
project_pickle_filename = os.environ.get('OQMBT_PROJECT')
oqtkp = OQtProject.load_from_file(project_pickle_filename)
model_id = oqtkp.active_model_id
model = oqtkp.models[model_id]

strain_pickle_spatial_index_filename = getattr(model, 'strain_pickle_spatial_index_filename')
strain_rate_model_hdf5_filename = getattr(model, 'strain_rate_model_hdf5_filename')

strain_rate_model_hdf5_filename = os.path.join(oqtkp.directory, strain_rate_model_hdf5_filename)
strain_pickle_spatial_index_filename = os.path.join(oqtkp.directory, strain_pickle_spatial_index_filename)

## Get strain data

In [ ]:
#
# Strain spatial index
strain_idx = index.Index(strain_pickle_spatial_index_filename)
#
# Strain data
f = h5py.File(strain_rate_model_hdf5_filename, 'r')
strain_data = f['gsrm'].value
f.close()

## Computing scalar moment from strain 

In [ ]:
#
#
shear_modulus = float(getattr(model, 'shear_modulus'))
cell_dx = model.strain_cell_dx
cell_dy = model.strain_cell_dy 
#
#
mo_dict = {}
for key in model.sources:
    source = model.sources[key]
    
    if re.match('AreaSource', source.source_type):
        #
        # set required parameters
        if 'coup_coef' in source.__dict__:
            coup_coef = source.coup_coef
        else:
            coup_coef = getattr(model, 'coup_coef')
        if 'coup_thick' in source.__dict__:
            coup_thick = source.coup_thick
        else:
            coup_thick = getattr(model, 'coup_thick')
        #
        # coupled seismogenic thickness
        coupl_seism_thickness = coup_thick * coup_coef * 1000
        #
        # computing the moment for the area source
        polygon = source.polygon 
        mo = get_moment_for_polygon(polygon, strain_data, strain_idx,
                                    coupl_seism_thickness, shear_modulus,
                                    cell_dx, cell_dy)
        source.mo_strain = mo
oqtkp.save()

In [ ]:
for key in model.sources:
    source = model.sources[key]
    if re.match('AreaSource', source.source_type):
        print(source.source_id, source.mo_strain)